In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

dataset = load_dataset("SKNahin/bengali-transliteration-data")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']


In [4]:
print(train_dataset)
print(val_dataset)

Dataset({
    features: ['bn', 'rm'],
    num_rows: 4004
})
Dataset({
    features: ['bn', 'rm'],
    num_rows: 1002
})


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")

tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "bn_IN"



In [6]:
def preprocess_function(examples):
    source = examples['rm']
    target = examples['bn']


    model_inputs = tokenizer(source, max_length=128, truncation=True, padding="max_length")


    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target, max_length=128, truncation=True, padding="max_length")["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs


In [7]:
train_tokenized = train_dataset.map(preprocess_function, batched=True)
val_tokenized = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [8]:
from transformers import MBartForConditionalGeneration


model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")


In [9]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-8d1e8cc006a1>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mdsabbir120834 (mdsabbir120834-ahsanullah-university-of-science-technology). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,13.777800,0.162415
2,0.550200,0.088232
3,0.270100,0.076222
4,0.092100,0.072197
5,0.052600,0.070460
6,0.034900,0.071673
7,0.024300,0.072014
8,0.012400,0.071017
9,0.009600,0.071655


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 261.06 MiB is free. Process 83657 has 14.49 GiB memory in use. Of the allocated memory 13.85 GiB is allocated by PyTorch, and 521.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
results = trainer.evaluate()
print(results)

Epoch,Training Loss,Validation Loss
1,13.777800,0.162415
2,0.550200,0.088232
3,0.270100,0.076222
4,0.092100,0.072197
5,0.052600,0.070460
6,0.034900,0.071673
7,0.024300,0.072014
8,0.012400,0.071017
9,0.009600,0.071192


{'eval_loss': 0.07119160890579224}
